# Ayudantía 7: Decoradores

### **Autores**: 
* Valentina Córdova
* Cristóbal Figueroa
* Sebastián Olivares
* Ignacio Urrutia

## Repaso

### Objetos
Los objetos son una colección de datos, a los que llamamos **atributos** (análogos a las variables) y comportamientos a los que llamamos **métodos** (análogos a las funciones). 

No nos detendremos mucho en su explicación porque se utilizarán a o largo del repaso, sin embargo, cualquier consulta, no duden en preguntar. 

### Excepciones

In [ ]:
# Creamos las clases de las excepciones personalizadas con una advertencia
class NoCorreo(Exception):
    def __init__(self, err):
        super().__init__(f"{err} no es un correo")


class NoCorreoUC(Exception):
    def __init__(self):
        super().__init__("No es un correo UC")


class CorreoRepetido(Exception):
    def __init__(self):
        super().__init__("Este correo ya existe en nuestra base de datos")


# Creamos la función que revisa todos los posibles errores y levanta los errores
def verificar_correo_uc(correo, basedatos):
    if '@' not in correo:                     # Verifica que sea un correo
        raise NoCorreo(correo)
    elif 'uc' not in correo.split('@')[1]:    # Verifica que sea un correo uc
        raise NoCorreoUC
    elif not correo.split('@')[0].isalnum():  # Verifica que sea alfanumerico
        raise TypeError
    elif correo in basedatos:                 # Verifica que no este en la base de datos
        raise CorreoRepetido


# Aquí tenemos el flujo donde tiene 3 intentos en total, se equivoque o no tiene 3 oportunidades
for intento in range(1, 4):
    try:
        archivo = open('db.csv', "r+")             # Abrimos el archivo
        db = [line.rstrip() for line in archivo]   # Quitamos los saltos de linea y hacemos la lista
        intento_correo = input("Ingrese su correo: ")
        verificar_correo_uc(intento_correo, db)
    # Aquí manejamos las excepciones avisando al usuario sobre sus errores
    except NoCorreo as error:
        print(f"Error: {error}. Este fue su intento {intento}, le quedan {3 - intento} intentos")
    except NoCorreoUC as error:
        print(f"Error: {error}. Este fue su intento {intento}, le quedan {3 - intento} intentos")
    except TypeError:
        print(f"Error: La primera parte debe ser alfanumerica. Este fue su intento {intento}, "
              f"le quedan {3 - intento} intentos")
    except CorreoRepetido as error:
        print(f"Error: {error}. Este fue su intento {intento}, le quedan {3 - intento} intentos")
    else:
        # Aquí guardamos los correos validos
        archivo.write('\n' + intento_correo)
        print(f"Se ha agregado correctamente. Puede agregar {3 - intento} "
              f"correos más, si no tiene errores")
    finally:
        archivo.close()   # Cerramos el archivo cada vez que terminamos de usarlo


### Threading

Crearemos una función que representa el movimiento de una persona, la cual imprime constantemente su avance de acuerdo a un tiempo de espera, de esta forma podemos tener personas que avancen más rápido que otras! 😮

In [ ]:
import random
import time

def movimiento_persona(n, sleep_time = 1):
  pos = 0
  for _ in range(n):
    # Mostramos nombre del thread
    thread_actual = threading.current_thread()
    # Seleccionamos movimienot aleatorio
    pos += 1
    print(f"Hola, soy {thread_actual.name} y he avanzado {pos} metros")
    time.sleep(sleep_time)

Ahora, utilizando threading, crearemos dos personas: 
1. Rapidín: Campeón internacional de 100 metros planos 
2. Lentín: Ayudante de programación avanzada después de pasar de largo corrigiendo la Tarea 1  

In [ ]:
# Importamos el módulo
import threading
import time

# Creamos threads
p1 = threading.Thread(name = "Rapidin", target = movimiento_persona, args=(10,0.4) )
p2 = threading.Thread(name = "Lentin", target = movimiento_persona, args=(10,1) )
# Ejecutamos el thread

p1.start() 
p2.start() # Recuerda que solo se pueden usar una vez si se definen de esta forma

p1.join()
print(f"Terminó {p1.name}")
if p2.is_alive(): 
 print("Aún falta la segunda persona")
 print(f"Esperando a {p2.name}...")
 p2.join()
 print("Terminaron las dos personas")
else:
 print("Proceso terminado, ambas terminaron a tiempo!")

Ahora creamos una clase `Contador` que servirá para almacenar los metros recorridos por Lentín y Rapidín.

In [ ]:
class Contador:
    def __init__(self):
        self.valor = 0

In [ ]:
import random
import time

def movimiento_persona(n, contador, lock, sleep_time = 1):
  pos = 0
  for _ in range(n):
    # Mostramos nombre del thread
    thread_actual = threading.current_thread()
    # Seleccionamos movimienot aleatorio
    pos += 1
    
    # Pedimos el lock con with
    with lock:
        contador.valor += 1
        print(f"Hola, soy {thread_actual.name}, he avanzado {pos} metros y en total llevamos {contador.valor} metros")

    # Pedimos el lock con acquire y release
#     lock.acquire()
#     contador.valor += 1
#     print(f"Hola, soy {thread_actual.name}, he avanzado {pos} metros y en total llevamos {contador.valor} metros")
#     # Liberamos el lock
#     lock.release()
    time.sleep(sleep_time)

In [ ]:
# Importamos el módulo
import threading

# Creamos el lock y contador
lock = threading.Lock()
contador = Contador()

# Creamos threads
p1 = threading.Thread(name = "Rapidin", target = movimiento_persona, args=(10, contador, lock,0.4) )
p2 = threading.Thread(name = "Lentin", target = movimiento_persona, args=(10, contador, lock) )

# Ejecutamos el thread
p1.start() 
p2.start() # Recuerda que solo se pueden usar una vez si se definen de esta forma

#Comentar desde aquí para probar con daemons
p1.join()
print(f"Terminó {p1.name}")
if p2.is_alive(): 
 print("Aún falta la segunda persona")
 print(f"Esperando a {p2.name}...")
 p2.join()
 print("Terminaron las dos personas")
else:
 print("Proceso terminado, ambas terminaron a tiempo!")

Como hacer para darle ventaja a Lentin?

Usaremos señales para comunicarle a Rapidín cuando Lentin vaya en la mitad!

#### Segunda forma:
Otra implementación de threading es mediante la herencia de la clase `Thread`

In [ ]:
import random
import time
import threading

medio_camino = threading.Event()

class Corredor(threading.Thread):
    def __init__(self,nombre, n, sleep_time, contador, lock):
        super().__init__(name = nombre)
        self.n = n
        self.contador = contador
        self.lock = lock
        self.sleep_time = sleep_time
        self.pos = 0
    
    def correr(self, n):
        for _ in range(n):
            # Mostramos nombre del thread
            thread_actual = threading.current_thread()
            # Seleccionamos movimienot aleatorio
            self.pos += 1
            # Pedimos el lock
            self.lock.acquire()
            self.contador.valor += 1
            print(f"Hola, soy {thread_actual.name}, he avanzado {self.pos} metros y en total llevamos {self.contador.valor} metros")
            # Liberamos el lock
            self.lock.release()
            time.sleep(self.sleep_time)

In [ ]:
class Rapidin(Corredor):
    def __init__(self, contador, lock):
        super().__init__("Rapidin", 10, .5, contador, lock)
    
    def run(self):
        # Esperamos la señal
        medio_camino.wait()
        # Y empezamos a correr
        self.correr(self.n)


class Lentin(Corredor):
    def __init__(self, contador, lock):
        super().__init__("Lentin", 10, 1, contador, lock)
    
    def run(self):
        # Empezamos a correr inmediatamente hasta la mitad
        self.correr(self.n//2)
        # Una vez en la mitad avisamos que hemos llegado
        medio_camino.set()
        self.correr( int( (self.n + 0.5)//2 ) )

In [ ]:
#Importamos el módulo
import threading

# Creamos el lock y contador
lock = threading.Lock()
contador = Contador()

# Creamos threads
p1 = Rapidin(contador, lock)
p2 = Lentin(contador, lock)


# Ejecutamos el thread
p1.start() 
p2.start() # Recuerda que solo se pueden usar una vez si se definen de esta forma


p1.join()
print(f"Terminó {p1.name}")
if p2.is_alive(): 
 print("Aún falta la segunda persona")
 print(f"Esperando a {p2.name}...")
 p2.join()
 print("Terminaron las dos personas")
else:
 print("Proceso terminado, ambas terminaron a tiempo!")

### Interfaces Gráfica

#### Nociones básicas

##### **Qapplication y Qwidget**
La base de toda aplicación con PyQt debe tener **siempre** una instancia de QApplication (¡Una! ni más, ni menos) y almenos una instancia de un QWidget (puede ser cualquier tipo de QWidget de los existentes,
como los ejemplos que les mostramos más adelante, incluyendo uno personalizado).

In [ ]:
import sys
from PyQt5.QtWidgets import QWidget, QApplication

if __name__ == '__main__':
    app = QApplication([])
    ventana = QWidget()
    ventana.show()
    sys.exit(app.exec_())

Se pueden ver los siguientes pasos en el código anterior:
- Importar los elementos necesarios.
- Instanciar una `QApplication` genérica. No es necesario editarla, por lo que siempre será similar a este ejemplo.
- Se crea una variable `ventana` que será una instancia de `QWidget`. Por último, utilizamos el método `.show()` (propio de todo `QWidget`) para mostrarla en pantalla
- La última línea es un comando común al utilizar `QApplications` y se asegura que Python termine una ejecución una vez se cierran todas las ventanas.

#### Ejemplo

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel, QLineEdit
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt, QTimer, QEventLoop

class VentanaConTodo(QWidget):

    def __init__(self, width, height, *args, **kwargs):
        ''' Se deben pasar los argumentos a la clase madre '''
        super().__init__(*args, **kwargs)
        self.size = (width, height)

        ''' Es buena práctica almacenar todo comando de inicio en un método similar '''
        self.init_gui()

    def init_gui(self):
        ''' Se configura el tamaño y título de la ventana '''
        self.setGeometry(400, 200, *self.size)
        self.setWindowTitle('Ventana con algunas cosas')

        # Comando para cambiar el stylesheet de algún objeto
        self.setStyleSheet("background-color: white")

        ''' Se crea personaje acompañador '''
        self.clip = QLabel(self)
        size_clip = [self.size[0] * 0.1, self.size[1] * 0.4] 
        self.clip.setGeometry(0, 0, *size_clip)
        #self.clip.setText("DCCIA")
        self.clip.setPixmap(QPixmap("clip.jpg"))
        self.clip.setScaledContents(True)

        ''' Se crea un editor de texto y boton de búsqueda'''
        self.buscador = QLineEdit(self)
        self.boton_buscador = QPushButton("Buscar", self)
        self.boton_buscador.clicked.connect(self.buscando)

        ''' Labels de instrucciones '''
        self.mostrador = QLabel(self)
        self.mostrador.setText("Aquí se mostrará lo escrito al buscar")
        self.instrucciones_clip = QLabel(self)
        self.instrucciones_clip.setText("Presiona W o clickea el clip para saltar")
        # self.instrucciones_clip.setText("A Clip le gustan los clicks y la W")

        ''' Se ordenan los elementos  de la parte izquierda'''
        self.instrucciones_clip.move(50, 10)
        self.clip.move(100, self.size[1] - size_clip[1] - 5)

        ''' Se ordenan los elementos  de la parte derecha'''
        self.mostrador.move(300, 90)
        self.buscador.move(320, 130)
        self.boton_buscador.move(340, 160)

        ''' Se muestra la ventana, si no se muestra aca podría mostrarse al instanciarla'''
        ''' Si no se hace ninguno no se mostrará '''
        self.show()

    ''' Esto se ejecutará cuando pulsemos el boton buscar '''
    def buscando(self):
        # Se extrae el texto del QLineEdit
        nuevo_texto = self.buscador.text()
        # Se muestra utilizando setText
        self.mostrador.setText(nuevo_texto)

    ''' Esto detectará cuando soltemos una tecla '''
    def keyReleaseEvent(self, event):

        ''' Lo usamos para hacer que el personaje salte'''
        ''' El evento detecta la tecla presionada, y se puede '''
        ''' comparar con objetos Key de Qt'''
        if event.key() == Qt.Key_W:
            self.saltar()
    
    ''' Detectará cuando se presione el mouse '''
    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton and \
            self.clip.x() < event.x() < self.clip.x() + self.clip.size().width() and \
            self.clip.y() < event.y() < self.clip.y() + self.clip.size().height():
            
            self.saltar()
    
    ''' Mueve nuestro acompañante con el comando move() '''
    def saltar(self):
        for _ in range(30):
            self.clip.move(self.clip.x(), self.clip.y() - 5)
            self.sleep(0.005)

        for _ in range(30):
            self.clip.move(self.clip.x(), self.clip.y() + 5)
            self.sleep(0.005)

    def sleep(self, secs):
        ''' Equivalente a time.sleep(secs) compatible con PyQt5'''
        loop = QEventLoop()
        QTimer.singleShot(secs * 1000, loop.quit)
        loop.exec_()



''' Esto ayuda a levantar excepciones inesperadas '''
def hook(type, value, traceback):
        print(type)
        print(traceback)
sys.__excepthook__ = hook


''' Importantísimo siemore instanciar con UNA QApplication '''
if __name__ == '__main__':
    app = QApplication([])
    ventana = VentanaConTodo(600, 300)
    sys.exit(app.exec_())

#### Frontend & Backend

Al diseñar un programa, es esencial tener la siguiente división:

* *Backend*: Parte encargada de la lógica del programa, cálculos, información que necesite ser almacenada, etc.

* *Frontend*: Parte encargada de mostrar los cambios que se vayan realizando en el *Backend*.

Esta disposición permite lograr estructuras mucho más complejas que lo que se podría lograr programando todo junto. Para poder comunicar apropiadamente estas estructuras son esenciales las *señales*.

Un par de cosas importantes de las señales:

* Comunmente se crean encima del \__init\__ de una clase, esto es para que sea un atributo global de la clase.
* Es necesario crear señales para cada sentido de comunicación, una del *backend* al *frontend* y otro en sentido contrario.
* Estas señales se deben conectar mediante el método *connect* de la siguiente manera: _clase_emisora.nombre_señal.connect(método_a_comenzar_al\_emitir)_ en el *main*.

### Properties

Una property funciona como un atributo, sobre el cual podemos modificar su comportamiento cada vez que es leído (get), escrito (set), o eliminado (del). Al usar el mecanismo de properties sobre un atributo, podemos ejecutar acciones de manera más limpia que invocando métodos explícitos para leer o modificar el valor de un objeto. 
Por supuesto debemos definir los métodos correspondientes getter y setter. Veremos que al utilizar estos métodos podemos agregar comportamiento adicional en cada caso.

PD: Los atributos cuyo comportamiento es modificado, no necesariamente son privados, de hecho, pueden ser públicos, privados o incluso no estar dentro de la misma clase en la cual se aplica la property.

In [ ]:
"""Property que mantiene los valores de las notas entre 1 y 7."""


class Alumno:
    """Clase para guardar nota de los alumnos."""

    def __init__(self, nombre, nota):
        """Init de la clase. Recibe nota cuando se crea."""
        self.nombre = nombre
        # self.__nota = nota
        self.nota = nota

    @property
    def nota(self):
        """Property. Es el getter de el atributo nota."""
        return self.__nota  # Como nota es ""privado" al tener doble guión bajo
                            # no se podría modificar sin un getter.
    @nota.setter
    def nota(self, nueva_nota): # Se recibe la NUEVA nota, no el cambio respecto al anterior
        if nueva_nota > 7:    # Aquí se verifica si se pasa de 7.
            self.__nota = 7
        elif nueva_nota < 1:  # Aquí se verifica si es menor que 1.
            self.__nota = 1
        else:                 # Aquí se pone la nota correspondiente si cumple los requisitos.
            self.__nota = nueva_nota

    def bonificacion(self, puntos):
        """Ejemplo de función que modifica el atributo."""
        self.nota += puntos
        print(f"La nota de {self.nombre} tras la bonificación es {self.nota}")

    def penalizacion(self, puntos):
        """Ejemplo de función que modifica el atributo."""
        self.nota -= puntos
        print(f"La nota de {self.nombre} tras la penalización es {self.nota}")

# EJEMPLO

# Se instancia un alumno con nota fuera de los rangos especificados
alumno_evaluado = Alumno("nacho_urrutia", 8.0)  # El alumno queda con nota 7 tras pasar por la property
print(f"La nota de {alumno_evaluado.nombre} es {alumno_evaluado.nota}")

alumno_evaluado.nota -= 1.5  # Puedes modificar la nota directamente gracias al setter
print(f"La nota de {alumno_evaluado.nombre} es {alumno_evaluado.nota}") 

# Aquí alumno_evaluado hizo excelente la tarea y ganó 2.5 ptos
alumno_evaluado.bonificacion(2.5)  # Esto sumaría 8 sin el setter

# Aquí alumno_evaluado no entregó el readme en la tarea y para hacerlo fácil le restamos 10.0
alumno_evaluado.penalizacion(10.0)  # Esto sumaría -3.0 sin el setter


## Contenidos de la Semana 9

### Funciones de primera clase
Python posee funciones de primera clase, lo cual significa que estas se pueden tratar tal como cualquier otra variable. Algunas consecuencias de aquello se muestran a continuación:

#### 1. Las funciones pueden ser asignadas a una variable, y luego usar esa variable igual que la función.

Referenciar funciones --> Puntero de funciones, si se modifica una se modifica la otra

In [ ]:
def suma(x, y):
    return x + y

adición = suma

# Ambas son la misma función
print(adición)
print(suma)

# Y por lo tanto entregan el mismo resultado
print(suma(3, 5))
print(adición(3, 5))

#### 2. Se pueden definir funciones anidadas, es decir, funciones dentro de otras funciones.

In [ ]:
def operacion(x, y):
    
    ## La función 'operacion_interna' se define DENTRO de 'operacion' y puede ser 
    ## usado dentro de ella
    def operacion_interna(z):
        return z ** 2
    
    ## Podemos usar 'operacion_interna' dentro de 'operacion'
    resultado = x + y + operacion_interna(x + y)
    return resultado

print(operacion(3, 5))

#### 3. Las funciones pueden ser pasadas como argumentos a otras funciones.


In [ ]:
def saludar_señora(nombre):
    return ' '.join(["Señora", nombre])

def saludar_señor(nombre):
    return ' '.join(["Señor", nombre])

## Recibe una función, y ejecuta un llamado con ella.
def saludar_tarde(función_saludo, nombre):
    return ' '.join(["Buenas tardes", función_saludo(nombre)])

## Aquí pasamos un nombre de función como argumento.
## Atención que no agregamos los '()' porque no estamos invocando a esa función
print(saludar_tarde(saludar_señora, "Valeria"))
print(saludar_tarde(saludar_señor, "Germán"))

#### 4. Las funciones pueden retornar otras funciones.

In [ ]:
def fabricar_funcion():
    ## Aquí definimos una función "dentro" de otra.
    ## Esto significa que el nombre 'nueva_funcion' solo es válido dentro 'fabricar_funcion'
    def nueva_funcion(x, y):
        return x * y
    
    print(f"Acabo de fabricar la función {nueva_funcion} y la retornaré")
    return nueva_funcion

## Este llamado no invoca a 'nueva_funcion', sino que solo la define y la retorna
funcion = fabricar_funcion()

## Ahora, 'funcion' queda definida como 'nueva_funcion'
print(f"funcion es {funcion}")
print(f"Invocando a funcion(3,5) --> {funcion(3,5)}")

#### 5. Las funciones definidas adentro de otras tienen acceso (sólo de lectura) a las variables del scope de la función que la contiene.

In [ ]:
def fabricar_funcion(x):
    texto = "Texto de prueba"
    print(f"[fabricar_funcion] Texto: {texto}")
    def nueva_funcion():
        texto = "Texto definitivo"
        print(f"[nueva_funcion] Texto: {texto}")
        return 2 * x
    print(f"[fabricar_funcion] Texto: {texto}")
    return nueva_funcion

In [ ]:
# Llamamos fabricar_función para obtener nuestra función que multiplica por dos
funcion = fabricar_funcion(3)

# Ahora, llamamos la función
print(funcion())

### Decoradores 😎
Los decoradores son una forma de agregarle funcionalidades a un objeto sin tener que reescribir su código. En este caso, utilizaremos decoradores de funciones, los cuales permiten tomar una función ya implementada, agregar alguna funcionalidad y retornar la función alterada.

Supongamos que queremos espamear a nuestro mejor amigo para que conteste el teléfono. Para eso utilizaremos dos funciones: `enviar_mensaje(amigo)` y `llamar(amigo)`

In [ ]:
def enviar_mensaje(amigo):
  print(f"Querid@ {amigo} por favor respóndeme")

def llamar(amigo):
  print(f"LLamando a {amigo}")

In [ ]:
llamar("CrisLucho")
enviar_mensaje("Seba Olivares")

Sin embargo, nos cansamos de mandar mensajes tantas veces a mano y decidimos automatizar este proceso para lo que podremos utilizar decoradores! :D 
Crearemos entonces un decorador `repetir_accion` que se encargará de repetir 10 veces la función decorada. Veamos como hacerlo!

In [ ]:
def repetir_accion(function):
  """
  La función wrapper 'envuelve' a la función que queremos alterar.
  Recibe n_spams para indicar la cantidad de veces a repetir
  la función que queremos manipular.
  """
  def wrapper(n_spams, nombre):
    for i in range(n_spams):
      function(nombre)
  return wrapper  # Retornamos la función modificada


In [ ]:
enviar_mensajes = repetir_accion(enviar_mensaje)
molestar = repetir_accion(llamar)

In [ ]:
enviar_mensajes(15,"CrisLucho")

Sin embargo, python tiene una forma más práctica de utilizar estos decoradores, y es anteponiedo y es de la siguiente manera:

In [ ]:
@repetir_accion
def enviar_mensaje(amigo):
  print(f"Yo: Querid@ {amigo} por favor respóndeme")

@repetir_accion
def llamar(amigo):
  print(f"LLamando a {amigo}...")

Probemos como funciona!

In [ ]:
llamar(35, "Vale Cordova")
enviar_mensaje(22, "Vale Cordova")

Pero que pasa si queremos utilizar nuestro decorador en funciones con distinto número de argumentos?...
Aquí llegan a salvarnos nuestros ~~no tan~~ queridos \*args y \**kwargs

In [ ]:
def repetir_accion(function):
  """
  La función wrapper 'envuelve' a la función que queremos alterar.
  Recibe *args y **kwargs para ser capaz de usar los mismos parámetros
  que la función que queremos manipular.
  """
  def wrapper(n_spams, *args, **kwargs):
    for i in range(n_spams):
      function(*args, **kwargs)
  return wrapper  # Retornamos la función modificada

In [ ]:
@repetir_accion
def enviar_mensaje(amigo, mensaje):
  print(f"Yo: Querid@ {amigo}, {mensaje}")

@repetir_accion
def llamar(amigo):
  print(f"LLamando a {amigo}...")

@repetir_accion
def enviar_objetos(amigo, direccion, lista_objetos):
  for objeto in (lista_objetos):
    print(f"{amigo} te ha llegado un/a {objeto}")
  print(f"Los objetos han sido enviados a {direccion}")

In [ ]:
a = "Nacho Urrutia"
llamar(8, a)
enviar_mensaje(10, a, "abre la puertaaa")
enviar_objetos(3, a, "Calle Falsa 123", ["huevo", "palta", "papel higiénico", "tomate"])


Finalmente, como los decoradores también son funciones, se les pueden entregar argumentos. En este caso, ingresaremos el número de repeticiones como argumento del decorador: 

In [ ]:
def repetir_accion(n_spam):
  def funcion_decorada(funcion):
    def wrapper(*args, **kwargs):
      for i in range(n_spam):
        funcion(*args, **kwargs)
    return wrapper  # Retornamos la función modificada
  return funcion_decorada


In [ ]:
@repetir_accion(10)
def enviar_mensaje(amigo, mensaje):
  print(f"Yo: Querid@ {amigo}, {mensaje}")

@repetir_accion(5)
def llamar(amigo):
  print(f"LLamando a {amigo}...")

@repetir_accion(3)
def enviar_objetos(amigo, direccion, lista_objetos):
  for objeto in (lista_objetos):
    print(f"{amigo} te ha llegado un/a {objeto}")
  print(f"Los objetos han sido enviados a {direccion}")

In [ ]:
llamar("Vale Cordova")
enviar_mensaje("Cris Lucho", "habla el jaguar, contesta")
enviar_objetos("Seba Olivares", "Jacarepagua 22302", ["DCCobranza","invitacion a DCCita","cubo de DCConsola"])

### Ejemplos

Imagina que volvemos a la Tarea 0 y necesitas ingresar un nombre y un apellido de usuario hasta que cumplan con el requerimiento de ser alfanuméricos, pero esta vez tienes tu as bajo la manga: **Decoradores**.  

In [ ]:
"""
Decorador encargado de pedir un input hasta que sea alfanumérico
"""

def alfanumerico(categoria):  # Aquí recibimos el nombre de la categoría (en este caso nombre o apellido)
  def funcion_decoradora(funcion):    # Recibimos la función sobre la cual actuará el decorador
    def wrapper(usuario):     # Recibimos los parámetros de la función anterior (al ser un método recibe como parámetro la clase a la que pertenece)
      ingresado = input(f"Ingresar {categoria}: ") # Pedimos el input
      while not ingresado.isalpha():  # Repetimos hasta que se cumplan las condiciones
        print("Debes ingresar solo caracteres alfanuméricos, inténtalo de nuevo!")
        ingresado = input(f"Ingresar {categoria}: ")
      return funcion(usuario, ingresado)  # Retornamos el resultado del método 
    return wrapper  # Retornamos la función modificada
  return funcion_decoradora

Ahora implementaremos nuestro decorador en una clase `Usuario` para asignarle un nombre y un apellido que cumplan con ser alfanuméricos.

In [ ]:
class Usuario:
  def __init__(self):
    self.asignar_nombre()
    self.asignar_apellido()
    self.saludar()
  
  @alfanumerico("apellido")
  def asignar_apellido(self, apellido):
    self.apellido = apellido
    
  @alfanumerico("nombre")
  def asignar_nombre(self, nombre):
    self.nombre = nombre

  def saludar(self):
    print(f"Bienvenid@ {self.nombre} {self.apellido}!")

In [ ]:
usuario = Usuario()